# Enrich Lane Frames with Lane Perception
---

Use **Advanced Computer Vision techniques** to determine the radius of the lane, calculate vehicle position with respect to center of the lane and draw lane boundary onto original image.

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
from pathlib import Path
import numpy as np
import pathlib
import sys
sys.path.append("/home/car/anaconda/envs/cv/lib/python3.7/site-packages")
import cv2
import os
import glob
import json
import pickle

In [ ]:
from lib.cv.CameraCalibration import CameraCalibration
from lib.cv.GradientThresholds import GradientThresholds
from lib.cv.ColorThresholds import ColorThresholds
from lib.cv.CameraPerspective import CameraPerspective
from lib.cv.LaneLineDetection import LaneLineDetection
from lib.cv.LaneLineCurvature import LaneLineCurvature
from lib.cv.LaneVehiclePosition import LaneVehiclePosition
from lib.cv.LaneBoundaries import LaneBoundaries

In [ ]:
import skvideo.io
import skvideo.datasets
from IPython.display import HTML
from fractions import Fraction
from tqdm import tqdm

%matplotlib inline

## Pipeline Algorithm

1\. **Camera Calibration**

Compute camera calibration matrix and distortion coefficients given a set of chessboard images using **CameraCalibration** class's `cmpt_mtx_and_dist_coeffs()` method.

2\. **Distortion Correction**

Apply distortion correction to raw images using **CameraCalibration** class's `correct_distortion()` method. First test undistortion on a raw chessboard image and visualize it. Then test undistortion on a raw curved lane line image and visualize it.

3\. **Color & Gradient Threshold**

Use color transforms, gradients, etc., to create a threshold binary image.

4\. **Perspective Transform**

Warp the image into a Bird's Eye View (also known as top down view).

5\. **Detect Lane Lines**

Locate which pixels are lane line pixels.

6\. **Determine Lane Curvature**

Calculate the left and right lane line's radius of curvature.

7\. **Determine Lane Center of Vehicle Position**

Calculate vehicle's position with respect to the lane center.

8\. **Display Lane Boundaries**

Visualized the warped lane boundaries back onto the undistorted image (original image right after distortion correction).

9\. **Display Lane Boundaries with Lane Curvature and Vehicle Position**

## Advanced Lane Pipeline (Image Processing)
---

Output images from each stage of the pipeline will be saved in folder `output_images`.

### Save Each Filepath and Filename into List For each Image

In [ ]:
# test_files[0] = [filepath, filename]
test_files = []
for filename in os.listdir("test_images/"):
    if filename.endswith(".jpg"):
        filepath = os.path.join("test_images/", filename)
        test_files.append([filepath, filename])

print(*test_files, sep = "\n")

## 1. Camera Calibration

Compute Camera Calibration Matrix and Distortion Coefficients 

In [ ]:
nx_corners = 9
ny_corners = 6
# Choose a directory with camera images of 9x6 chessboards 720x1280p(h x w)
cam_cal_chess_dfp = "camera_cal/calibration*.jpg"
smart_car_cam = CameraCalibration(nx_corners, ny_corners, cam_cal_chess_dfp)

# Compute camera calibration matrix and distortion coefficients via a set of chessboard images
src_img_fp = "camera_cal/calibration3.jpg"
cal_mtx, dist_coeff = smart_car_cam.cmpt_mtx_and_dist_coeffs(src_img_fp)

## 2. Distortion Correction

Apply Distortion Correction to Raw Images

### 2A. Visualize Corrected Distortion on a Raw Chessboard Image

In [ ]:
# Apply Distortion Correction to a raw chessboard image
smart_car_cam.set_dist_img(src_img_fp)
chess_dist_img, chess_undist_img = smart_car_cam.correct_distortion(cal_mtx, dist_coeff)

# Save Undistorted Chessboard Image
dst_img_fp = "data/output/image/pipeline/distortion_correction/"
filename = "calibration3.jpg"
smart_car_cam.save_img(dst_img_fp, filename, chess_undist_img, cal_mtx, dist_coeff)

# Visualize Chessboard Image from Distorted and Undistorted 
smart_car_cam.visualize(filename, chess_dist_img, filename, chess_undist_img)

### 2B. Visualize Corrected Distortion of each Lane Line Image

In [ ]:
lane_dist_img_list = []
lane_undist_img_list = []

for file in test_files:
    # First element of each row is filepath
    # Apply Distortion Correction to each distort lane line image
    src_img_fp = file[0]
    smart_car_cam.set_dist_img(src_img_fp)
    lane_dist_img, lane_undist_img = smart_car_cam.correct_distortion(cal_mtx, dist_coeff)
    
    print(type(lane_undist_img), lane_undist_img.shape)
    
    lane_dist_img_list.append(lane_dist_img)
    lane_undist_img_list.append(lane_undist_img)
    
    # Visualize Chessboard Image from Distorted and Undistorted 
    smart_car_cam.visualize(file[1], lane_dist_img, file[1], lane_undist_img)
    
    # Save Each Undistorted lane line image using filepath
    dst_img_fp = "data/output/image/pipeline/distortion_correction/"
    smart_car_cam.save_img(dst_img_fp, file[1], lane_undist_img, cal_mtx, dist_coeff)

## 3. Color & Gradient Threshold

Create a Thresholded Binary Image using Color & Gradient Thresold

In [ ]:
gradient_car_cam = GradientThresholds()

### 3A. Use Gradient Thresholds to Create Binary Image

### Apply X-Sobel Orientation Derivative Thresholding

Identifies pixels where the gradient of an image falls within a specified threshold range. 
Can also take the Y-Sobel, but found X-Sobel to be better at picking up lane lines.

In [ ]:
# Applies Gradient Thresholding to Undistorted Car Camera Images
orient = 'x'
thresh_min = 20 # 20
thresh_max = 100 # 100

# List for saving sobel-x
sx_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    sx_binary_img = gradient_car_cam.apply_sobel_thresh(lane_undist_img, orient, thresh_min, thresh_max)

    # Append sobel-x binary image to list
    sx_binary_img_list.append(sx_binary_img)
    
    # Visualizes a figure with undistorted image and gradient thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'X-Sobel: ' + file[1]
    gradient_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, sx_binary_img)
    
    # Save each binary image resulting from gradient thresholding
    dst_img_fp = "data/output/image/pipeline/gradient_threshold/x_sobel/"
    gradient_car_cam.save_img(dst_img_fp, file[1], sx_binary_img)

### Apply Both X and Y Gradient Magnitude Thresholding

This thresholding can be done over larger regions to smooth over noisy intensity 
fluctuations on small scales

In [ ]:
# Applies Gradient Magnitude Thresholding to Undistorted Lane Img
sobel_kernel = 3
mag_thresh = (20, 100)

# List for saving gradient magnitude of undistorted lane images
grad_mag_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    grad_mag_binary_img = gradient_car_cam.apply_grad_mag_thresh(lane_undist_img, sobel_kernel, mag_thresh)
    
    # Append gradient magnitude binary image to list
    grad_mag_binary_img_list.append(grad_mag_binary_img)
    
    # Visualizes a figure with undistorted image and gradient magnitude thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Grad Magn: ' + file[1]
    gradient_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, grad_mag_binary_img)
    
    # Save each binary image resulting from gradient thresholding
    dst_img_fp = "data/output/image/pipeline/gradient_threshold/gradient_magnitude/"
    gradient_car_cam.save_img(dst_img_fp, file[1], grad_mag_binary_img)    

### Apply Gradient Direction Thresholding

This thresholding is useful for picking up edges of a particular orientation.

In [ ]:
sobel_kernel = 15
dir_thresh = (0.7, 1.3) # 0.7, 1.3

# List for saving gradient direction of undistorted lane images
grad_dir_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    grad_dir_binary_img = gradient_car_cam.apply_grad_dir_thresh(lane_undist_img, sobel_kernel, dir_thresh)
    
    # Append gradient direction binary image to list
    grad_dir_binary_img_list.append(grad_dir_binary_img)
    
    # Visualizes a figure with undistorted image and gradient direction thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Grad. Dir: ' + file[1]
    gradient_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, grad_dir_binary_img)
    
    # Save each binary image resulting from gradient thresholding
    dst_img_fp = "data/output/image/pipeline/gradient_threshold/gradient_direction/"
    gradient_car_cam.save_img(dst_img_fp, file[1], grad_dir_binary_img)    

### Apply Combined Gradient Thresholds

Use various aspects of gradient measurements (x, y, magnitude, direction) to isolate lane-line pixels. First, I did a selection for pixels where x and y gradients meet the threshold criteria or the gradient magnitude and direction are both within their threshold values.

In [ ]:
# List for saving combined gradient of undistorted lane images
comb_grad_binary_img_list = []

for lane_undist_img, sx_binary_img, grad_mag_binary_img, grad_dir_binary_img, file in zip(lane_undist_img_list, sx_binary_img_list, grad_mag_binary_img_list, grad_dir_binary_img_list, test_files):
    # apply_combined_thresh has combination codes:
    # 0: X-Sobel, Gradient Magnitude
    # 1: X-Sobel, Gradient Direction
    # 2: X-Sobel, Gradient Magnitude, Gradient Direction
    # 3: X-Sobel, Y-Sobel, Gradient Magnitude, Gradient Direction    
    comb_grad_binary_img = gradient_car_cam.apply_combined_thresh(2, 
                                        grad_x = sx_binary_img,
                                        grad_mag = grad_mag_binary_img,
                                        grad_dir = grad_dir_binary_img)

    # Append combined gradient binary image to list
    comb_grad_binary_img_list.append(comb_grad_binary_img)
    
    # Visualizes a figure with undistorted image and combined gradient thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Comb. Grad: ' + file[1]
    gradient_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, comb_grad_binary_img)
    
    # Save each binary image resulting from gradient thresholding
    dst_img_fp = "data/output/image/pipeline/gradient_threshold/combined_gradient/"
    gradient_car_cam.save_img(dst_img_fp, file[1], comb_grad_binary_img)    

### 3B. Use Color Thresholds to Create Binary Image

In [ ]:
color_car_cam = ColorThresholds()

### Apply Grayscale Thresholding

Convert to grayscale and apply a threshold to identify lane lines

In [ ]:
thresh = (180, 255)

# List for saving gray binary resulting from undistorted lane images
gray_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    gray_binary_img = color_car_cam.apply_gray_thresh(lane_undist_img, thresh)
    
    # Append gradient direction binary image to list
    gray_binary_img_list.append(gray_binary_img)
    
    # Visualizes a figure with undistorted image and grayscale thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Grayscale: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, gray_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/grayscale/"
    color_car_cam.save_img(dst_img_fp, file[1], gray_binary_img)

### Apply Invidual RGB Thresholding

Using Red, Green and Blue to identify **white lane line pixels**.

#### Apply R Thresholding

R channel does a reasonable job of highlighting lines, so you can apply threshold to find lane-lines

In [ ]:
r_thresh = (130, 255) # (130, 255) best so far

# List for saving red binary image
red_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    red_binary_img = color_car_cam.apply_r_thresh(lane_undist_img, r_thresh)
    
    # Append gradient direction binary image to list
    red_binary_img_list.append(red_binary_img)
    
    # Visualizes a figure with undistorted image and grayscale thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'R Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, red_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/rgb/red/"
    color_car_cam.save_img(dst_img_fp, file[1], red_binary_img)

#### Apply G Thresholding

Use G channel to highlight lines and apply threshold to find lane-lines

In [ ]:
g_thresh = (130, 255) # (130, 255) best so far

# List for saving green binary image
green_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    green_binary_img = color_car_cam.apply_g_thresh(lane_undist_img, g_thresh)
    
    # Append gradient direction binary image to list
    green_binary_img_list.append(green_binary_img)
    
    # Visualizes a figure with undistorted image and grayscale thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'G Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, green_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/rgb/green/"
    color_car_cam.save_img(dst_img_fp, file[1], green_binary_img)

#### Apply B Thresholding

Use B channel to highlight lines and apply threshold to find lane-lines

In [ ]:
b_thresh = (195, 255) # (185, 255) best so far

# List for saving green binary image
blue_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    blue_binary_img = color_car_cam.apply_b_thresh(lane_undist_img, b_thresh)
    
    # Append blue binary image to list
    blue_binary_img_list.append(blue_binary_img)
    
    # Visualizes a figure with undistorted image and blue thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'B Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, blue_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/rgb/blue/"
    color_car_cam.save_img(dst_img_fp, file[1], blue_binary_img)

### Apply Combined RGB Thresholding

Combining RGB (Red, Green, Blue) to identify white lane pixels

In [ ]:
# List for saving combined rgb binary images
comb_rgb_binary_img_list = []

for lane_undist_img, red_binary_img, green_binary_img, blue_binary_img, file in zip(lane_undist_img_list, red_binary_img_list, green_binary_img_list, blue_binary_img_list, test_files):
    # combination codes:
    # 0: R Binary, G Binary
    # 1: R Binary, B binary
    # 2: G Binary, B Binary
    # 3: R Binary, G Binary, B Binary   
    comb_rgb_binary_img = color_car_cam.apply_rgb_thresh(3, 
                                        rgb_r = red_binary_img,
                                        rgb_g = green_binary_img,
                                        rgb_b = blue_binary_img)
    
    # Append combined rgb binary image to list
    comb_rgb_binary_img_list.append(comb_rgb_binary_img)
    
    # Visualizes a figure with undistorted image and combined gradient thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Comb. RGB: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, comb_rgb_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/rgb/comb_rgb/"
    color_car_cam.save_img(dst_img_fp, file[1], comb_rgb_binary_img)    

### Apply Individual HLS Thresholding

Combining HLS (Hue, Lightness, Saturation) to identify **yellow lane line pixels**.

#### Apply H Thresholding for Detecting Yellow

The lane lines appear dark in the H channel, so we try a low threshold and obtain the following result

In [ ]:
h_thresh = (20, 90) # (20, 90) best so far, detects warm red to yellow green lane line pixels

# List for saving hue binary image
hue_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    hue_binary_img = color_car_cam.apply_h_thresh(lane_undist_img, h_thresh)
    
    # Append saturation binary image to list
    hue_binary_img_list.append(hue_binary_img)
    
    # Visualizes a figure with undistorted image and saturation thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'H Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, hue_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/hls/hue/"
    color_car_cam.save_img(dst_img_fp, file[1], hue_binary_img)

#### Apply L Thresholding for Detecting Yellow

L channel picks up the lines well, so let's try applying a threshold.

In [ ]:
# Apply Threshold to pick up yellow colors
l_thresh = (120, 200) # (215, 255) good at picking up white lane line pixels

# List for saving lightness binary image
light_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    light_binary_img = color_car_cam.apply_l_thresh(lane_undist_img, l_thresh)
    
    # Append saturation binary image to list
    light_binary_img_list.append(light_binary_img)
    
    # Visualizes a figure with undistorted image and saturation thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'L Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, light_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/hls/lightness/"
    color_car_cam.save_img(dst_img_fp, file[1], light_binary_img)

#### Apply S Thresholding for Detecting Yellow

S channel picks up the lines well, so let's try applying a threshold.

In [ ]:
# Apply Threshold to pick up shades of yellow colors
s_thresh = (100, 255)

# List for saving saturation binary image
sat_binary_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    sat_binary_img = color_car_cam.apply_s_thresh(lane_undist_img, s_thresh)
    
    # Append saturation binary image to list
    sat_binary_img_list.append(sat_binary_img)
    
    # Visualizes a figure with undistorted image and saturation thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'S Binary: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, sat_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/hls/saturation/"
    color_car_cam.save_img(dst_img_fp, file[1], sat_binary_img)

### Combine HLS - Hue, Lightness, Saturation

Right now, I have HLS set to combine Lightness and Saturation.

In [ ]:
# Save combined hls binary images
comb_all_hls_binary_img_list = []

for lane_undist_img, hue_binary_img, light_binary_img, sat_binary_img, file in zip(lane_undist_img_list, hue_binary_img_list, light_binary_img_list, sat_binary_img_list, test_files):
    # combination codes:
    # 0: H Binary, L Binary
    # 1: H Binary, S binary
    # 2: L Binary, S Binary
    # 3: H Binary, L Binary, S Binary
    comb_all_hls_binary_img = color_car_cam.apply_hls_thresh(3, 
                                        hls_h = hue_binary_img,
                                        hls_l = light_binary_img,
                                        hls_s = sat_binary_img)  
    # Append combined hls binary image to list
    comb_all_hls_binary_img_list.append(comb_all_hls_binary_img)
    
    # Visualizes a figure with undistorted image and combined gradient thresholded image
    src_title = 'Undistorted: ' + file[1]
    thresh_img_title = 'Comb. HLS: ' + file[1]
    color_car_cam.visualize(src_title, lane_undist_img, thresh_img_title, comb_all_hls_binary_img)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/color_threshold/hls/comb_all_hls/"
    color_car_cam.save_img(dst_img_fp, file[1], comb_all_hls_binary_img)        

### Color Thresholding Summary

From the color thresholding above for RGB and HSL, you can see 
H channel is probably your best bet. It's near the same result
as S channel and a bit better than the R channel or simple grayscaling.

### Combine Color & Gradient Thresholding

We will see which parts of the lane lines were detected by the gradient threshold and which parts were detected by color threshold by stacking the channels and seeing the individual components.

We will create a binary combination of these two images to map out where either the color or gradient thresholds were met.

### Combine Gradient Sobel X and HLS Saturation

In [ ]:
# List for saving combined Sobel-X and Saturation binary images
comb_sx_sat_binary_img_list = []
# List for saving stacked Sobel-X and Saturation color images
stack_sx_sat_color_img_list = []

for lane_undist_img, sx_binary_img, sat_binary_img, file in zip(lane_undist_img_list, sx_binary_img_list, sat_binary_img_list, test_files):
    # Stack Scaled Sobel X and S Color Channel:
    # to view their individual contributions in green and blue respectively
    # Green color is sobel-x gradient threshold component
    # Blue color is saturation threshold component 
    sx_sat_color_binary = np.dstack(( np.zeros_like(sx_binary_img), sx_binary_img, sat_binary_img )) * 255

    # Append stacked Sobel-X and Saturation color image to list
    stack_sx_sat_color_img_list.append(sx_sat_color_binary)
    
    # Combine the two binary thresholds
    comb_sx_sat_binary_img = np.zeros_like(sx_binary_img)
    comb_sx_sat_binary_img[ (sx_binary_img == 1) | (sat_binary_img == 1) ] = 1

    # Append combined Sobel-X and Saturation binary image to list
    comb_sx_sat_binary_img_list.append(comb_sx_sat_binary_img)
    
    # Visualizes a figure with undistorted image and combined gradient thresholded image
    src_title = 'Stack. S-X_SAT: ' + file[1]
    thresh_img_title = 'Comb. S-X_SAT: ' + file[1]
    gradient_car_cam.visualize(src_title, sx_sat_color_binary, thresh_img_title, comb_sx_sat_binary_img)
    
    # Save each stacked color image
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/stacked_sobelx_sat/"
    gradient_car_cam.save_img(dst_img_fp, file[1], sx_sat_color_binary)
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/combined_sobelx_sat/"
    gradient_car_cam.save_img(dst_img_fp, file[1], comb_sx_sat_binary_img)

### Combine HLS and RGB

HLS identified yellow lane line pixels and RGB identified white lane line pixels.

In [ ]:
# List for saving combined hls, rgb binary images
comb_hls_rgb_binary_img_list = []
# List for saving stacked Sobel-X and Saturation color images
stack_hls_rgb_color_img_list = []

for lane_undist_img, comb_all_hls_binary_img, comb_rgb_binary_img, file in zip(lane_undist_img_list, comb_all_hls_binary_img_list, comb_rgb_binary_img_list, test_files):
    # Stack HLS and RGB Color Channel:
    # to view their individual contributions in green and blue respectively
    # Green color is HLS threshold component
    # Blue color is RGB threshold component
    hls_rgb_color_binary = np.dstack(( np.zeros_like(comb_all_hls_binary_img), comb_all_hls_binary_img, comb_rgb_binary_img )) * 255

    # Append stacked HLS, RGB color image to list
    stack_hls_rgb_color_img_list.append(hls_rgb_color_binary)
    
    # Combine the three (HLS, RGB) binary thresholds
    comb_hls_rgb_binary_img = np.zeros_like(comb_all_hls_binary_img)
    comb_hls_rgb_binary_img[ (comb_all_hls_binary_img == 1) |
                             (comb_rgb_binary_img == 1) ] = 1

    # Append combined HLS, RGB binary image to list
    comb_hls_rgb_binary_img_list.append(comb_hls_rgb_binary_img)
    
    # Visualizes a figure with undistorted image and combined HLS + RGB binary image
    stack_title = 'HLS_RGB: ' + file[1]
    thresh_title = 'HLS_RGB: ' + file[1]
    gradient_car_cam.visualize(stack_title, hls_rgb_color_binary, thresh_title, comb_hls_rgb_binary_img)
    
    # Save each stacked HLS + RGB binary image
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/stacked_hls_rgb/"
    gradient_car_cam.save_img(dst_img_fp, file[1], hls_rgb_color_binary)
    
    # Save each binary image resulting from HLS + RGB binary image
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/combined_hls_rgb/"
    gradient_car_cam.save_img(dst_img_fp, file[1], comb_hls_rgb_binary_img)

### Combined Gradient with RGB and HLS

In [ ]:
# List for saving combined gradient, rgb, hls binary images
comb_grad_hls_rgb_binary_img_list = []
# List for saving stacked gradient, rgb, hls color images
stack_grad_hls_rgb_color_img_list = []

for comb_grad_binary_img, comb_hls_rgb_binary_img, file in zip(comb_grad_binary_img_list, comb_hls_rgb_binary_img_list, test_files):
    # Stack Combined Gradient, RGB, HLS Color Channel:
    # to view their individual contributions in green and blue respectively
    # Green color is combined Gradient threshold component
    # Blue color is RGB, HLS threshold component
    stack_grad_hls_rgb_color_img = np.dstack(( np.zeros_like(comb_grad_binary_img), comb_grad_binary_img, comb_hls_rgb_binary_img )) * 255

    # Append stacked combined Gradient, HLS, RGB color image to list
    stack_grad_hls_rgb_color_img_list.append(stack_grad_hls_rgb_color_img)
    
    # Combine the three (Gradient, HLS, RGB) binary thresholds
    comb_grad_hls_rgb_binary_img = np.zeros_like(comb_grad_binary_img)
    comb_grad_hls_rgb_binary_img[ (comb_grad_binary_img == 1) |
                                  (comb_hls_rgb_binary_img == 1) ] = 1

    # Append combined Gradient, HLS, RGB binary image to list
    comb_grad_hls_rgb_binary_img_list.append(comb_grad_hls_rgb_binary_img)
    
    # Visualizes a figure with undistorted image and combined HLS + RGB binary image
    stack_title = 'Grad_HLS_RGB: ' + file[1]
    thresh_title = 'Grad_HLS_RGB: ' + file[1]
    gradient_car_cam.visualize(stack_title, stack_grad_hls_rgb_color_img, thresh_title, comb_grad_hls_rgb_binary_img)
    
    # Save each stacked Gradient, HLS + RGB binary image
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/stacked_grad_hls_rgb/"
    gradient_car_cam.save_img(dst_img_fp, file[1], stack_grad_hls_rgb_color_img)
    
    # Save each binary image resulting from HLS + RGB binary image
    dst_img_fp = "data/output/image/pipeline/gradient_color_threshold/combined_grad_hls_rgb/"
    gradient_car_cam.save_img(dst_img_fp, file[1], comb_grad_hls_rgb_binary_img)

## 4. Perspective Transform

Identify four source points in an image where the lane lines are straight. After perspective transform, make the lines look straight and vertical from a bird's eye view perspective.

In [ ]:
car_cam_view = CameraPerspective()

### Change Camera View to Bird's Eye View

### 4A. Show Source Points

In [ ]:
# List for saving source points used in Bird's Eye View
mark_src_pts_img_list = []

for lane_undist_img, file in zip(lane_undist_img_list, test_files):
    imshape = lane_undist_img.shape
        
#     plt.imshow(lane_undist_img)
#     plt.plot(imshape[1]*0.145, imshape[0], '.', markersize = 12) # bottom left
#     plt.plot(imshape[1]*0.462, imshape[0]*0.62, '.', markersize = 12) # top left
#     plt.plot(imshape[1]*0.535, imshape[0]*0.62, '.', markersize = 12) # bottom right
#     plt.plot(imshape[1]*0.883, imshape[0], '.', markersize = 12) # bottom right

    # Notice there are two straight lane line images, those are my
    # two base images for choosing source points in warping image
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(lane_undist_img, cmap = 'gray')
    ax1.set_title("Undistorted: " + file[1], fontsize=30)
    ax2.imshow(lane_undist_img, cmap = 'gray')
    ax2.set_title("Source Pts: " + file[1], fontsize=30)
    ax2.plot(imshape[1]*0.145, imshape[0], '.', markersize = 25) # bottom left
    ax2.plot(imshape[1]*0.462, imshape[0]*0.62, '.', markersize = 25) # top left
    ax2.plot(imshape[1]*0.535, imshape[0]*0.62, '.', markersize = 25) # bottom right
    ax2.plot(imshape[1]*0.883, imshape[0], '.', markersize = 25) # bottom right    
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/perspective_transform/source_points/"
    car_cam_view.save_fig(dst_img_fp, file[1])

### 4B. Warp Lane Images to Bird's Eye View

Uses Combined Sobel-X, Hue, Saturation Binary image

In [ ]:
# List for saving lane line images warped to Bird's Eye View
warped_lane_bev_img_list = []

for comb_grad_hls_rgb_binary_img, file in zip(comb_grad_hls_rgb_binary_img_list, test_files):
    warped_lane_bev_img = car_cam_view.birds_eye_view(comb_grad_hls_rgb_binary_img)

    # Add Bird's Eye View lane line images
    warped_lane_bev_img_list.append(warped_lane_bev_img)
    
    # Visualizes a figure with combined gradient and color thresholded image
    # with warped birds eye view image
    src_title = 'Grad_HLS_RGB: ' + file[1]
    warp_img_title = 'Warped. BEV: ' + file[1]
    car_cam_view.visualize(src_title, comb_grad_hls_rgb_binary_img, warp_img_title, warped_lane_bev_img)    
    
    # Save each binary image resulting from warped birds eye view
    dst_img_fp = "data/output/image/pipeline/perspective_transform/birds_eye_view/"
    car_cam_view.save_img(dst_img_fp, file[1], warped_lane_bev_img)

### 4C. Show Destination Points for Warped Images

In [ ]:
# List for saving destination points used in Bird's Eye View
mark_dst_pts_img_list = []

for comb_grad_hls_rgb_binary_img, warped_lane_bev_img, file in zip(comb_grad_hls_rgb_binary_img_list, warped_lane_bev_img_list, test_files):
    imshape = comb_grad_hls_rgb_binary_img.shape

    # Notice there are two straight lane line images, those are my
    # two base images for choosing source points in warping image
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(comb_grad_hls_rgb_binary_img, cmap = 'gray')
    ax1.set_title("Source Pts: " + file[1], fontsize=30)
    ax1.plot(imshape[1]*0.145, imshape[0], '.', markersize = 25) # bottom left
    ax1.plot(imshape[1]*0.462, imshape[0]*0.62, '.', markersize = 25) # top left
    ax1.plot(imshape[1]*0.535, imshape[0]*0.62, '.', markersize = 25) # bottom right
    ax1.plot(imshape[1]*0.883, imshape[0], '.', markersize = 25) # bottom right      
    
    ax2.imshow(warped_lane_bev_img, cmap = 'gray')
    ax2.set_title("Destn Pts: " + file[1], fontsize=30)
    ax2.plot(imshape[1]*0.24, imshape[0], '.', markersize = 25) # bottom left
    ax2.plot(imshape[1]*0.24, imshape[0]*0, '.', markersize = 25) # top left
    ax2.plot(imshape[1]*0.75, imshape[0]*0, '.', markersize = 25) # bottom right
    ax2.plot(imshape[1]*0.75, imshape[0], '.', markersize = 25) # bottom right
    
    # Save each binary image resulting from color thresholding
    dst_img_fp = "data/output/image/pipeline/perspective_transform/destination_points/"
    car_cam_view.save_fig(dst_img_fp, file[1])

## 5. Detect Lane Lines

### 5A. Histogram Peaks

Now that we have the thresholded warped image, we can map out the lane lines! There are multiple ways to go about doing this, we'll use a **histogram** to see where the binary activations (peaks) occur across the image, so we can decide explicitly which pixels are part of the lines and which belong to the left line and which belong to the right line.

In [ ]:
# Create an instance of the finding lane lines class
find_lane_lines = LaneLineDetection()

# Histogram list for saving histograms for each lane line image
histogram_list = []

# Select either to visualize only histogram or histogram and lane lines
only_histogram = False
lane_bev_and_histogram = True

for warped_lane_bev_img, file in zip(warped_lane_bev_img_list, test_files):
    # Create histogram of image binary activations
    histogram = find_lane_lines.histogram_peaks(warped_lane_bev_img)

    # Append histogram to list
    histogram_list.append(histogram)
    
    # Titles to use in the below visualization
    warp_img_title = 'Warped. BEV: ' + file[1]
    hist_title = 'Histogram: ' + file[1]
    if only_histogram == True and lane_bev_and_histogram == True:
        # Visualize only the resulting histogram
        find_lane_lines.visualize_hist(hist_title, histogram)
        # Save only histogram plotted from birds eye view lane line image
        dst_img_fp = "data/output/image/pipeline/detect_lane_lines/histogram/"
        find_lane_lines.save_fig(dst_img_fp, file[1])
        # show figure of only histogram separately
        plt.show()
        
        # Visualize the lane line birds eye view image and histogram
        find_lane_lines.visualize_lanes_and_hist(
            warp_img_title, 
            warped_lane_bev_img,
            hist_title,
            histogram)
        # Save only histogram plotted from birds eye view lane line image
        dst_img_fp = "data/output/image/pipeline/detect_lane_lines/lanes_and_hist/"
        find_lane_lines.save_fig(dst_img_fp, file[1])
        # show figure of lane birds eye view next to histogram separately
        plt.show()    
    elif only_histogram == True:
        # Visualize only the resulting histogram
        find_lane_lines.visualize_hist(hist_title, histogram)
        # Save only histogram plotted from birds eye view lane line image
        dst_img_fp = "data/output/image/pipeline/detect_lane_lines/histogram/"
        find_lane_lines.save_fig(dst_img_fp, file[1])
    elif lane_bev_and_histogram == True:
        # Visualize the lane line birds eye view image and histogram
        find_lane_lines.visualize_lanes_and_hist(
            warp_img_title, 
            warped_lane_bev_img,
            hist_title,
            histogram)
        # Save only histogram plotted from birds eye view lane line image
        dst_img_fp = "data/output/image/pipeline/detect_lane_lines/lanes_and_hist/"
        find_lane_lines.save_fig(dst_img_fp, file[1])
    else:
        print("None of the options were seleted")

### 5B. Sliding Windows

We use sliding windows moving upward in the image (further along the road) to determine where the lane lines go, essentially we are finding the lane line pixels.

In [ ]:
# Store lane line imgs with sliding windows and fitted polynomial
out_img_list = []
# Store left fit polynomial in pixels
left_fitx_list = []
# Store right fit polynomial in pixels
right_fitx_list = []

for warped_lane_bev_img, histogram, file in zip(warped_lane_bev_img_list, histogram_list, test_files):
    # Uses Histogram peaks and Sliding Window method to find all pixels
    # belonging to each line (left and right line)
    out_img = find_lane_lines.find_lane_pixels(warped_lane_bev_img, histogram)
    
    # Fits a polynomial to each line in pixels
    left_fitx, right_fitx = find_lane_lines.fit_polynomial(warped_lane_bev_img)
    
    # Append left fit polynomial in pixels to left line
    left_fitx_list.append(left_fitx)
    # Append right fit polynomial in pixels to right line
    right_fitx_list.append(right_fitx)

    # Visualize the sliding windows and fit polynomial per line
    fig_title = "Sliding Windows: " + file[1]
    find_lane_lines.visualize_fit_polynomial(out_img, left_fitx, right_fitx, fig_title)
    
    # Append lane line imgs with sliding windows and fitted polynomial
    out_img_list.append(out_img)
    
    # Save lane line sliding window images
    dst_img_fp = "data/output/image/pipeline/detect_lane_lines/sliding_windows/"
    find_lane_lines.save_img(dst_img_fp, file[1], out_img)
    
    # Save lane line images with sliding windows and fit polynomials
    dst_img_fp = "data/output/image/pipeline/detect_lane_lines/fit_polynomials/"
    find_lane_lines.save_fig(dst_img_fp, file[1])
    
    # Show each figure individually
    plt.show()

### 5C. Search from Prior

We've built an algorithm that uses sliding windows to track the lane lines out into the distance. Yet, using the algorithm everytime to start fresh on every frame may seem inefficient since the lines don't necessarily move alot from frame to frame. So, in the next frame of video, we can just search in a margin around the previous line position. Thus, once we know where the lines are in one frame of video, we can do a highly targeted search for them in the next frame. This is equivalent to using a customized region of interest for each frame of video and should help with tracking the lanes through sharp cuves and tricky conditions. If we lose track of the lines, we can go back to sliding windows search to rediscover them.

In [ ]:
# Store search from prior left fit polynomial in pixels
left_fitx_sfp_list = []
# Store search from prior right fit polynomial in pixels
right_fitx_sfp_list = []

for warped_lane_bev_img, left_fitx, right_fitx, file in zip(warped_lane_bev_img_list, left_fitx_list, right_fitx_list, test_files):
    # Search from prior polynomials to get left_fitx and right_fitx
    left_fitx, right_fitx = find_lane_lines.search_around_poly(warped_lane_bev_img)
    find_lane_lines.visualize_sap(warped_lane_bev_img, left_fitx, right_fitx)
    
    left_fitx_sfp_list.append(left_fitx)
    right_fitx_sfp_list.append(right_fitx)
    
    # Save lane line images with search from prior fit polynomials
    dst_img_fp = "data/output/image/pipeline/detect_lane_lines/search_from_prior/"
    find_lane_lines.save_fig(dst_img_fp, file[1])   
    
    # Show individual figures
    plt.show()

### Sliding Windows vs Search From Prior

Notice the sliding windows compared to search from prior polynomial takes the last fit polynomial to lane line pixels and uses it to place on new frames with lane lines. When the search from prior approach doesnt match well with the actual direction of lane lines, then sliding windows approach needs to be applied again to find the lane line pixels, then replot a new polynomial to each line, so the lane lines are accurately detected.

In [ ]:
# Use both Sliding Windows and Search from Prior Techniques
# 1. If first time detecting lane lines, then use Sliding Windows

# 2. If second or more times, then try Search from Prior
# 3. If Search from Prior technique is too far off from actual lane
# lines, then re-use Sliding Windows to get the best fit polynomial 
# to each line

## 6. Measure Lane Line Curvature

We have polynomial fits and can calculate the radius of curvature. We will use LaneLineDetection class's **fet_fit_polynomial_data()** to pass the needed data to LaneLineCurvature class's **measure_curvature_pixels()**. The method uses Rcurve equation:

### Second Order Polynomial

Lane line pixels were located and their x and y pixel positions were used to fit a second order polynomial curve:

f(y) = Ay^2 + By + C

> Note: f(y) was fit rather than f(x) since the lane lines in the warped image are near vertical.

### Radius of Curvature

The radius of curvature at any point x of the function x = f(y):

Rcurve = ([1 + (dx/dy)^2]^(3/2))/(|(d^(2)x)/(dy^(2))|)

In the case of the second order polynomial, the 1st and 2nd derivatives are:

f'(y) = (dx)/(dy) = 2Ay + B
f''(y) = (d^(2)x)/(dy^(2)) = 2A

So, our equation for radius of curvature:

**Rcurve** = ((1 + (2Ay + B)^2)^(3/2))/(|2A|)

### Lane Line Curvature from Pixels to Real-World

Converting our x and y values to real world space involves measuring how long and wide the section of lane is we're projecting in our warped image. **We can do this by measuring out the physical lane in the field of view of the camera.** Yet, for this project, we can assume if we're projecting a section of lane similar to the images above, the lane is about 30 meters long and 3.7 meters wide.

> Note: If you prefer to derive a conversion from pixel space to world space in your own images, compare your images with U.S. regulations that require a minimum lane width of 12 feet or 3.7 meters and the dashed lane lines are 10 feet or 3 meters long each.

### Conversion Pixel space to Real-World Meters

Our camera image has 720 pixels in the y-dimension
and 700 pixels in the x-dimension. 200 pixels on the left to 900 on the right were used, so 900-200 = 700 pixels.

~~~python
ym_per_pix = 30/720 # meters per pixel in y dimension

xm_per_pixel = 3.7/700 # meters per pixel in x dimension
~~~

In [ ]:
calc_lane_curve = LaneLineCurvature()

In [ ]:
# Store left line curvature to list
left_curverad_list = []
# Store right line curvature to list
right_curverad_list = []
# Store lane line curvature units
lane_curverad_units = None

# Applying Sliding Windows first, then calculate lane curvature
for warped_lane_bev_img, histogram, file in zip(warped_lane_bev_img_list, histogram_list, test_files):
    # Uses Histogram peaks and Sliding Window method to find all pixels
    # belonging to each line (left and right line)
    out_img = find_lane_lines.find_lane_pixels(warped_lane_bev_img, histogram)
    
    # Fits a polynomial to each line in pixels
    find_lane_lines.fit_polynomial(warped_lane_bev_img)
    
    # Get data from each fit polynomial
    ploty, left_fit, right_fit = find_lane_lines.get_fit_polynomial_data()  
    
    # Measures Lane Line Curvature
    left_curverad, right_curverad, curverad_units = calc_lane_curve.measure_radius_curvature(ploty, left_fit, right_fit, "meters")
    
    # Store left line curvature to list
    left_curverad_list.append(left_curverad)
    # Store right line curvature to list
    right_curverad_list.append(right_curverad)
    # Store lane line curvature units to list
    lane_curverad_units = curverad_units
    
    # Displays radius of curvature
    frame_title = file[1]
    calc_lane_curve.display_radius_curvature(frame_title)    

## 7. Determine Lane Center of Vehicle Position

Calculate the vehicle's position with respect to lane center.

In [ ]:
lane_vehicle = LaneVehiclePosition()

# Store ploty
ploty_list = []
# Store left fit polynomial in pixels
left_fit_list = []
# Store right fit polynomial in pixels
right_fit_list = []

# Store vehicle's distance from the center of the lane to list
dist_center_list = []
# Store vehicle's position units
vehicle_position_units = None
# Store if vehicle's position is left or right of the center
side_center_list = []

# Applying Sliding Windows first, then calculate lane curvature
for warped_lane_bev_img, histogram, file in zip(warped_lane_bev_img_list, histogram_list, test_files):
    # Uses Histogram peaks and Sliding Window method to find all pixels
    # belonging to each line (left and right line)
    out_img = find_lane_lines.find_lane_pixels(warped_lane_bev_img, histogram)
    
    # Fits a polynomial to each line in pixels
    find_lane_lines.fit_polynomial(warped_lane_bev_img)
    
    # Get data from each fit polynomial
    ploty, left_fit, right_fit = find_lane_lines.get_fit_polynomial_data()

    # Append each ploty to list
    ploty_list.append(ploty)
    # Append each left fit polynomial to list
    left_fit_list.append(left_fit)
    # Append each right fit polynomial to list    
    right_fit_list.append(right_fit)    
    
    # Measure Vehicle's position with respect to lane center
    dist_center, position_units, side_center = lane_vehicle.measure_vehicle_position(warped_lane_bev_img, left_fit, right_fit, "meters")

    # Append vehicle's distance from center of the lane
    dist_center_list.append(dist_center)
    
    # Save vehicle's distance from lane center in units
    vehicle_position_units = position_units
    
    # Append string of whether vehicle is left or right of lane center
    side_center_list.append(side_center)
    
    # Print Vehicle's Position
    frame_title = file[1]    
    lane_vehicle.display_vehicle_position(frame_title)

## 8. Display Lane Boundaries

In [ ]:
lane_boundary = LaneBoundaries()

# Store images with lane boundary overlayed to list
lane_boundary_img_list = []

# Overlay lane boundary onto warped lane images, then display them
for lane_undist_img, warped_lane_bev_img, histogram, ploty, left_fit, right_fit, file in zip(lane_undist_img_list, warped_lane_bev_img_list, histogram_list, ploty_list, left_fit_list, right_fit_list, test_files):
    # Set binary image curved_lane_td in lane_boundary
    lane_boundary.set_warped_binary_img(warped_lane_bev_img)

    # Set undistorted image lane_undist_img
    lane_boundary.set_original_undist_img(lane_undist_img)

    # Set attributes related to fit the lines with a polynomial
    lane_boundary.set_fit_lines_poly(ploty, left_fit, right_fit)

    # Get inverse perspective matrix
    Minv = car_cam_view.get_minv()

    # Set inverse perspective matrix (Minv)
    lane_boundary.set_minv(Minv)

    # Overlay Detected Lane Boundaries back onto original image
    lane_boundary.overlay_lane_boundaries()

    # Visualize the overlay lane boundaries
    lane_boundary.visualize()
    # Get overlayed boundary image result
    lane_boundary_img = lane_boundary.get_overlayed_image()
    
    # Append lane image with lane boundary overlayed
    lane_boundary_img_list.append(lane_boundary_img)
    
    # Save lane images overlayed with lane boundary highlighted
    dst_img_fp = "data/output/image/pipeline/lane_perception/lane_boundary/"
    lane_boundary.save_img(dst_img_fp, file[1], lane_boundary_img)

## 9. Display Lane Boundaries with Lane Curvature and Vehicle Position

In [ ]:
# Sets text properties to configure how text is displayed in image
font_family=cv2.FONT_HERSHEY_SIMPLEX
font_color=(255,255,255)
font_size=1.6
font_thickness = 3
line_type = cv2.LINE_AA
lane_boundary.set_img_text_properties(font_family, font_color, font_size, font_thickness, line_type)

### Overlay Lane Curvature

In [ ]:
# Store lane images with lane boundary and lane curvature overlayed
lane_bound_curve_img_list = []

# Overlay lane boundary onto warped lane images, then display them
for lane_boundary_img, left_curverad, right_curverad, file in zip(lane_boundary_img_list, left_curverad_list, right_curverad_list, test_files):
    # Set the overlayed lane boundary image to be used by lane_boundary
    lane_boundary.set_overlayed_lane_boundary(lane_boundary_img)

    # Sets left and right lane curvature radius and units
    lane_boundary.set_lane_curvature_radius(left_curverad, right_curverad, lane_curverad_units)

    # Adds Lane Curvature Radius text to lane boundaries image
    lane_boundary.overlay_lane_curvature()

    # Visualizes the lane boundaries image with lane curvature added
    lane_boundary.visualize()
    
    # Get overlayed lane boundary and curvature image result
    lane_bound_curve_img = lane_boundary.get_overlayed_image()
    
    # Append lane image with lane boundary and curvature overlayed
    lane_bound_curve_img_list.append(lane_bound_curve_img)
    
    # Save lane images overlayed with lane boundary highlighted
    dst_img_fp = "data/output/image/pipeline/lane_perception/lane_curvature/"
    lane_boundary.save_img(dst_img_fp, file[1], lane_bound_curve_img)    

### Overlay Vehicle Position

In [ ]:
# Store lane images with lane boundary, lane curvature 
# and vehicle position with respect to lane center overlayed
lane_perception_img_list = []

# Overlay lane boundary onto warped lane images, then display them
for lane_bound_curve_img, dist_center, side_center, file in zip(lane_bound_curve_img_list, dist_center_list, side_center_list, test_files):
    # Set the overlayed lane boundary with curvature image to be used by lane_boundary
    lane_boundary.set_overlayed_lane_boundary(lane_bound_curve_img)

    # Set Vehicle Position with respect to center of lane
    lane_boundary.set_vehicle_position(dist_center, vehicle_position_units, side_center)

    # Adds Vehicle Position text to lane boundaries image
    lane_boundary.overlay_vehicle_position()

    # Visualizes the lane boundaries image with vehicle position added
    lane_boundary.visualize()    
    
    # Get overlayed lane boundary, lane curvature and vehicle position
    # image result
    lane_perception_img = lane_boundary.get_overlayed_image()
    
    # Append lane image with lane boundary, lane curvature 
    # and vehicle position overlayed
    lane_perception_img_list.append(lane_perception_img)
    
    # Save lane images overlayed with lane boundary highlighted
    dst_img_fp = "data/output/image/pipeline/lane_perception/vehicle_position/"
    lane_boundary.save_img(dst_img_fp, file[1], lane_perception_img)

## Advanced Lane Pipeline (Video Processing)
---

Modify each frame from video for **lane perception** by drawing **lane 
boundary**, **radius of lane curvature** and **vehicle position with respect to center of lane**.

I must test my pipeline on video:

- `project_video.mp4`

In [ ]:
# run_pipeline detects radius of lane curvature, vehicle position 
# with respect to center of lane and fills in the lane boundary
# over the original undistorted image
def run_pipeline(frame):
    ##
    # 1. Camera Calibration
    ##
    nx_corners = 9
    ny_corners = 6
    
    # Choose a directory with camera images of 9x6 chessboards 720x1280p(h x w)
    cam_cal_chess_dfp = "camera_cal/calibration*.jpg"
    calibrate_cam = CameraCalibration(nx_corners, ny_corners, cam_cal_chess_dfp)

    # Compute camera calibration matrix and distortion coefficients via a set of chessboard images
    src_img_fp = "camera_cal/calibration3.jpg"
    cal_mtx, dist_coeff = calibrate_cam.cmpt_mtx_and_dist_coeffs(src_img_fp)
    
    ##
    # 2. Image Distortion Correction
    ##
    
    # Apply Distortion Correction to a raw lane line frame
    dist_frame, undist_frame = calibrate_cam.correct_distortion(cal_mtx, dist_coeff, frame)

    ##
    # 3. Color & Gradient Thresholding
    ##
    
    # Applies X-Sobel Gradient Threshold to Undistorted Camera Frame
    orient = 'x'
    thresh_min = 20
    thresh_max = 100
    gradient_cam = GradientThresholds()
    sx_binary_frame = gradient_car_cam.apply_sobel_thresh(undist_frame, orient, thresh_min, thresh_max)   
    
    # Applies S Threshold to Undistorted Camera Frame
    # since it more easily identifies bright color lane lines
    color_cam = ColorThresholds()
    s_thresh = (100, 255)
    s_binary_frame = color_cam.apply_s_thresh(undist_frame, s_thresh)

    # Combine X-Sobel and S Thresold
    # Stack Scaled Sobel X and S Color Channel:
    # to view their individual contributions in green and blue respectively
    xs_color_binary = np.dstack(( np.zeros_like(sx_binary_frame), sx_binary_frame, s_binary_frame )) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sx_binary_frame)
    combined_binary[ (s_binary_frame == 1) | (sx_binary_frame == 1) ] = 1

    ##
    # 4. Perspective Transform to Bird Eye View
    ##
    
    cam_view = CameraPerspective()
    
    # Change Lane Line Frame to Bird's Eye View, still binary frame
    lane_b_e_view = cam_view.birds_eye_view(combined_binary)  

    ##
    # 5. Detect Lane Lines (Enhanced)
    ##
    
    # Create an instance of the finding lane lines class
    find_lane_lines = LaneLineDetection()

    # First use histogram peaks to determine where lane lines start:
    # Create histogram of image binary activations
    histo = find_lane_lines.histogram_peaks(lane_b_e_view)
    
    # Second use Sliding Window method to find all lane pixels:
    # belonging to each line (left and right line)
    out_img = find_lane_lines.find_lane_pixels(lane_b_e_view, histo)

    # Fit a polynomial to each line in pixels (left_fitx, right_fitx)
    left_fitx, right_fitx = find_lane_lines.fit_polynomial(lane_b_e_view)

    # Third use Search from Prior Polynomial for future frames:
    # to avoid re-compute teadious sliding window method
#     left_fitx, right_fitx = find_lane_lines.search_around_poly(lane_b_e_view)   

    ##
    # 6. Measure the Radius of Lane Curvature
    ##
    
    calc_lane_curve = LaneLineCurvature()

    # Get data from fit polynomial
    ploty, left_fit, right_fit = find_lane_lines.get_fit_polynomial_data()

    # Measures Lane Line Curvature
    left_curverad, right_curverad, curverad_units = calc_lane_curve.measure_radius_curvature(ploty, left_fit, right_fit, "meters")

    ##
    # 7. Determine Vehicle's Position with Respect to Lane Center
    ##
    
    lane_vehicle = LaneVehiclePosition()

    # Measure Vehicle's position with respect to lane center
    dist_center, position_units, side_center = lane_vehicle.measure_vehicle_position(lane_b_e_view, left_fit, right_fit, "meters")

    ##
    # 8. Overlay Lane Boundaries onto Undistorted Frame (Like Original)
    ##
    
    lane_boundary = LaneBoundaries()

    # Set binary frame lane_b_e_view in lane_boundary
    lane_boundary.set_warped_binary_img(lane_b_e_view)

    # Set undistorted frame undist_frame
    lane_boundary.set_original_undist_img(undist_frame)

    # Set attributes related to fit the lines with a polynomial
    lane_boundary.set_fit_lines_poly(ploty, left_fit, right_fit)

    # Get inverse perspective matrix
    Minv = cam_view.get_minv()

    # Set inverse perspective matrix (Minv)
    lane_boundary.set_minv(Minv)

    # Overlay Detected Lane Boundaries back onto undistorted frame
    lane_boundary.overlay_lane_boundaries()
    
    ##
    # 9. Add Lane Curvature and Vehicle Position onto Undist Frame
    ##
    
    # Sets text properties to configure how text is displayed in image
    font_family=cv2.FONT_HERSHEY_SIMPLEX
    font_color=(255,255,255)
    font_size=1.6
    font_thickness = 3
    line_type = cv2.LINE_AA
    lane_boundary.set_img_text_properties(font_family, font_color, font_size, font_thickness, line_type)    
    
    # Sets left and right lane curvature radius and units
    lane_boundary.set_lane_curvature_radius(left_curverad, right_curverad, curverad_units)

    # Adds Lane Curvature Radius text to lane boundaries image
    lane_boundary.overlay_lane_curvature()
    
    # Set Vehicle Position with respect to center of lane
    lane_boundary.set_vehicle_position(dist_center, position_units, side_center)

    # Adds Vehicle Position text to lane boundaries image
    lane_boundary.overlay_vehicle_position()
    
    # Result of overlayed frame with lane boundaries, lane curvature
    # and vehicle position
    result = lane_boundary.get_overlayed_image()
    
    return result

### Apply Advanced Lane Finding Pipeline to Video Stream

Uses **[scikit-video](www.scikit-video.org/stable/index.html)** to enrich each frame in the video for lane perception.

How long will it take to enrich the video?
3:00PM - 4:33PM

### Get Input Video Metadata for Writing Output Modified Video

Use **scikit-video** to handle gathering input video metadata.

In [ ]:
video_input_path = "project_video.mp4"

# See Input Video Metadata, Its needed for writing Output Video
metadata = skvideo.io.ffprobe(video_input_path)
print(metadata.keys())
print(json.dumps(metadata["video"], indent=4))

### Write Output Video with Frames Modified

Use **OpenCV** to handle reading input video frame by frame. Then use **OpenCV** to handle writing each modified frame to output video.

Video Processing takes about **1 hour (15 - 25) minutes**.

In [ ]:
# Specify the output filename with its format
video_mod_out_path = "data/output/video/project_video_mod.mp4"

# Specify video metadata for JSON Path Extraction
video_metadata = metadata["video"]

# Grab FourCC (video codec) based on input video: h264
fourcc_name = video_metadata["@codec_name"]

# Grab the number of frames per second (FPS) based on input video
frames_per_sec = int(Fraction(video_metadata["@avg_frame_rate"]))

# Specify frame size based on input video
frame_width = int(video_metadata["@width"])
frame_height = int(video_metadata["@height"])

# Create a VideoWriter object (OpenCV)
video_writer = cv2.VideoWriter(video_mod_out_path, cv2.VideoWriter_fourcc(*fourcc_name), frames_per_sec, (frame_width, frame_height))

# Create a VideoReader object (Scikit-Video)
# video_reader = skvideo.io.vreader(video_input_path)

video_reader = cv2.VideoCapture(video_input_path)

# Get total number of frames based on input video
total_frames = int(video_metadata["@nb_frames"])
    
# Loop while there are frames left to read from video_reader    
for i in tqdm(range(total_frames)):
    ret, frame = video_reader.read()
    if ret == True:
        # Modifying frame for lane perception by drawing lane boundary,
        # radius of lane curvature and vehicle position with respect to
        # center of lane
        mod_frame = run_pipeline(frame)        
        
        # Write the frame into the file 'project_video_mod.mp4'
        video_writer.write(mod_frame)
    # Break the loop, video_reader.read() didn't return frame
    else:
        break

# When everything done, release video write object
video_writer.release()

### Embed Video Mod Clip Output

In [ ]:
HTML("""
<video width="{1}" height="{2}" controls>
    <source src="{0}">
</video>
""".format(video_mod_out_path, frame_width, frame_height))